# Project_One

## In this project, we will be using Covid-19-API to analyze Covid related data around the world.

### Please refer to the documentation here: 
- https://github.com/M-Media-Group/Covid-19-API


## Analysis Goals:

### Part I: Live cases data
https://covid-api.mmediagroup.fr/v1/cases


### Part II: Historical cases data
https://covid-api.mmediagroup.fr/v1/history


### Part III: Vaccines data
https://covid-api.mmediagroup.fr/v1/vaccines

### Part IV: Make your own search
* I'm also thinking using 'input' function to allow users to enter the countries they would like to search and put them into a list. 
* This can be separated from the first three parts of analysis. Then, we can show the data in a map using Google Geocode, Google Places and gmaps, including the information as a marker layer in gmaps and the number as a heat layer.


## For each part I - III, we will go through the following analysis in individual jupyter notebook:

1. Collect data from API and put the data into one csv file. You can use the above links as the base_url.<br /> 
   **There are different parameters we can choose from for each endpoint. We probably want to narrow down to a specific list of countries or status to make our own stories! 

2. Clean up the data. Only save the columns of interest and drop any NA. May also need extra columns, eg. percentage of cases over total population etc.

3. Generate plots and perform statistical analysis.

4. A summary of findings.

## Extra notes:

* There is no api_key required for Covid-19-API. However, after the initial data analysis, we may be incorporating our data and show on the gmaps using google Geocode and google Places. So I added the config.py to .gitignore to hide our keys.

* Below is a starter code that may help, but feel free to start in your own ways.

* I will incude this part as a README.md in our repositories Project_One. I grant access to everyone, you can change the content and I will merge the edits from everyone. Seara and I already have a branch in this repo :)<br /> 
<span style="color:red">Please send me your username on github. You will receive an invitation from me.</span>

* After you get access to the repo, you can do the following to make a directory in your local workspace and add your branch:
- *git clone git@github.com:ZiboZhao0804/project_one.git*
- *git checkout -b YOUR_NAME_HERE*
- *git push --set-upstream origin YOUR_NAME_HERE*

*<span style="color:blue"> We can decide how we would like to proceed with "division of labor" later in the class or anytime via Slack. Since we have four people in our team, I'm thinking one part per person , so that all of us can have some pratice of the workflow above. The four parts can be done in parallel, and after that, we can work together to put all together. </span>

*<span style="color:blue"> We can also come up with a timeline for our tasks so that we have enough time to put everything into a final presentation.</span>

In [1]:
import os
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
from config import g_key

In [ ]:
!wget https://raw.githubusercontent.com/M-Media-Group/country-json/master/src/countries-master.json

In [19]:
with open('countries-master.json','r') as f:
    countries_data = json.load(f)

In [22]:
countries_df = pd.DataFrame(countries_data)
countries_df.head()

,country,population,sq_km_area,life_expectancy,elevation_in_meters,continent,abbreviation,location,iso,capital_city
0,Afghanistan,35530081,652090.0,45.9,None,Asia,AF,Southern and Central Asia,4,Kabul
1,Albania,2930187,28748.0,71.6,None,Europe,AL,Southern Europe,8,Tirana
2,Algeria,41318142,2381741.0,69.7,800,Africa,DZ,Northern Africa,12,Alger
3,American Samoa,55641,199.0,75.1,None,Oceania,AS,Polynesia,16,Fagatogo
4,Andorra,76965,468.0,83.5,"1,996",Europe,AD,Southern Europe,20,Andorra la Vella


In [4]:
case_base_url = 'https://covid-api.mmediagroup.fr/v1/cases'
case_parameter = '?country=US'
case_data = requests.get(case_base_url+case_parameter).json()
pprint(case_data)

{'Alabama': {'confirmed': 585607,
             'deaths': 11536,
             'lat': '32.3182',
             'long': '-86.9023',
             'recovered': 0,
             'updated': '2021/08/01 00:21:33+00'},
 'Alaska': {'confirmed': 75486,
            'deaths': 389,
            'lat': '61.370716',
            'long': '-152.404419',
            'recovered': 0,
            'updated': '2021/08/01 00:21:33+00'},
 'All': {'abbreviation': 'US',
         'capital_city': 'Washington',
         'confirmed': 34971597,
         'continent': 'North America',
         'country': 'US',
         'deaths': 613133,
         'elevation_in_meters': 760,
         'iso': 840,
         'life_expectancy': '77.1',
         'location': 'North America',
         'population': 324459463,
         'recovered': 0,
         'sq_km_area': 9363520},
 'Arizona': {'confirmed': 925169,
             'deaths': 18224,
             'lat': '33.729759',
             'long': '-111.431221',
             'recovered': 0,
        

In [7]:
history_base_url = 'https://covid-api.mmediagroup.fr/v1/history'
history_parameter = '?status=Confirmed&country=US'
history_data = requests.get(history_base_url+history_parameter).json()
pprint(history_data)

{'All': {'abbreviation': 'US',
         'capital_city': 'Washington',
         'continent': 'North America',
         'country': 'US',
         'dates': {'2020-01-22': 1,
                   '2020-01-23': 1,
                   '2020-01-24': 2,
                   '2020-01-25': 2,
                   '2020-01-26': 5,
                   '2020-01-27': 5,
                   '2020-01-28': 5,
                   '2020-01-29': 6,
                   '2020-01-30': 6,
                   '2020-01-31': 8,
                   '2020-02-01': 8,
                   '2020-02-02': 8,
                   '2020-02-03': 11,
                   '2020-02-04': 11,
                   '2020-02-05': 11,
                   '2020-02-06': 12,
                   '2020-02-07': 12,
                   '2020-02-08': 12,
                   '2020-02-09': 12,
                   '2020-02-10': 12,
                   '2020-02-11': 13,
                   '2020-02-12': 13,
                   '2020-02-13': 14,
                   '2020-0

In [8]:
vaccines_base_url = 'https://covid-api.mmediagroup.fr/v1/vaccines'
vaccines_parameter = '?country=US'
vaccines_data = requests.get(vaccines_base_url+vaccines_parameter).json()
pprint(vaccines_data)

{'All': {'abbreviation': 'US',
         'administered': 345640466,
         'capital_city': 'Washington',
         'continent': 'North America',
         'country': 'US',
         'elevation_in_meters': 760,
         'iso': 840,
         'life_expectancy': '77.1',
         'location': 'North America',
         'people_partially_vaccinated': 190982149,
         'people_vaccinated': 164446964,
         'population': 324459463,
         'sq_km_area': 9363520,
         'updated': '2021/08/01 00:00:00+00'}}
